In [7]:
from utils import get_embedding, LlamaEmbeddingFunction
import os

In [8]:
chapter_files = sorted(os.listdir('./data/harry_potter/chapters'))
chapter_ids = list(map(lambda x: x.split('.')[0].replace(" ", "_"), chapter_files[:17]))

In [9]:
#load all chapters into memory 
chapters = []
for chapter_file in chapter_files[:17]:
    chapter = open(f'data/harry_potter/chapters/{chapter_file}', 'r').read()
    chapters.append(chapter)

--------

ChromaDB

In [10]:
import chromadb

In [11]:
client = chromadb.PersistentClient(path="chroma/harry_potter_chapters")

In [ ]:
# checks whether collection exists
if not client.has_collection("harry_potter_chapters"):
    collection = client.create_collection(name="harry_potter_chapters", embedding_function=LlamaEmbeddingFunction(), metadata={"hnsw:space": "cosine"})

In [12]:

collection = client.get_collection(name="harry_potter_chapters", embedding_function=LlamaEmbeddingFunction())

In [13]:
metadatas = []
for id in chapter_ids:
    c_number = id.split("_")[0]
    c_name = " ".join(id.split("_")[1:])
    metadatas.append({"chapter_number": c_number, "chapter_name": c_name})

In [14]:
collection.add(
    documents=chapters,
    metadatas=metadatas,
    ids=chapter_ids,
)

-------

Querying

In [15]:
question = "Who imagines himself holding socks when looking at the mirror of Erised ?"
q_embedding = get_embedding(question)
collection.query(
    query_embeddings=q_embedding,
    n_results=9,
    include=["documents", "distances"],
    #where={"metadata_field": "is_equal_to_this"},
    #where_document={"$contains":"sock"}
)

{'ids': [['008_the_potions_master',
   '016_through_the_trapdoor',
   '014_norbert_the_norwegian_ridgeback',
   '015_the_forbidden_forest',
   '011_quidditch_as_they_entered_november_,_the_weather_turned_very_cold',
   '002_the_vanashig_glass',
   '017_the_man_with_two_faces',
   '007_the_sorting_hat',
   '006_the_journey_from_platform_nine_and_three_quarters']],
 'distances': [[0.7805783827877196,
   0.7913916764128326,
   0.8074978494090049,
   0.8098711639965375,
   0.8124825679605856,
   0.8135776921851919,
   0.8149830811747119,
   0.8280079987290053,
   0.8459256160124947]],
 'metadatas': None,
 'embeddings': None,
 'documents': [['Chapter : THE POTIONS MASTER .\n “ There , look . ”\n “ Where ? ”\n “ Next to the tall kid with the red hair . ”\n “ Wearing the glasses ? ”\n “ Did you see his face ? ”\n “ Did you see his scar ? ”\n Whispers followed Harry from the moment he left his dormitory the next day .\n People lining up outside classrooms stood on tiptoe to get a look at him ,